In [14]:
import pickle
from collections import defaultdict

import pandas as pd
import numpy as np
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise.model_selection import KFold
from surprise import Dataset
from surprise import Reader

In [2]:
reader = Reader(sep=',', rating_scale=(1,10))

data = Dataset.load_from_file('../Data/ratings_compressed.csv', reader=reader)

In [3]:
train, test = train_test_split(data, test_size=0.3, random_state=42)
options = {'name':'cosine'}
knnModel = KNNBasic(sim_options=options)

In [4]:
knnModel.fit(train)
predictions = knnModel.test(test)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [5]:
n = 10

top_n = defaultdict(list)
for uid, iid, true_r, est, _ in predictions:
    top_n[uid].append((iid, est))

for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n[uid] = user_ratings[:n]

top_n

defaultdict(list,
            {'1718': [('1968', 8.069183801261941),
              ('2', 7.898387313763459),
              ('661', 7.730943474226144),
              ('1631', 7.151021564940655),
              ('1618', 6.877448085868441),
              ('1684', 6.374815261269697),
              ('1414', 5.992956169309171),
              ('1792', 5.978590719776129),
              ('591', 5.733707299174481),
              ('753', 3.955733071400593)],
             '977': [('826', 9.59950346882115),
              ('796', 9.325),
              ('1997', 8.524029984986644),
              ('706', 7.401165671018549),
              ('702', 7.275),
              ('316', 6.925),
              ('747', 6.715960272717734),
              ('523', 6.598240145042823),
              ('293', 6.52368570566486),
              ('30', 6.375)],
             '91': [('1142', 9.7),
              ('223', 9.65035902942506),
              ('219', 8.886535365778075),
              ('915', 8.868392847242387),
           

In [6]:
user_id = 1
item_id = 1
prediction = knnModel.predict(user_id, item_id)

In [7]:
test2 = pd.read_csv('../Data/test.csv', names=['user_id', 'profile_id', 'rating'])
test2['user_id'] = test2['user_id'].astype(str)
test2['profile_id'] = test2['profile_id'].astype(str)

tuples = [tuple(x) for x in test2.to_numpy()]

In [8]:
predictions = knnModel.test(tuples)

In [12]:
# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 2.0198


2.019823515221608

In [15]:
f = open('model/KNN.pickle', 'wb')
pickle.dump(knnModel, f)  
f.close()

In [16]:
f = open('model/KNN.pickle','rb') 
loaded_model = pickle.load(f)
f.close()

In [17]:
predictions = loaded_model.test(tuples)

In [18]:
# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 2.0198


2.019823515221608